In [3]:
!pip install max==25.4.0 --index-url https://dl.modular.com/public/nightly/python/simple/

Looking in indexes: https://dl.modular.com/public/nightly/python/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.0/285.0 MB 3.0 MB/s eta 0:00:00


In [4]:
!git clone https://github.com/modular/mojo-gpu-puzzles

Cloning into 'mojo-gpu-puzzles'...
remote: Enumerating objects: 6332, done.
remote: Counting objects: 100% (481/481), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 6332 (delta 448), reused 416 (delta 416), pack-reused 5851 (from 3)
Receiving objects: 100% (6332/6332), 148.65 MiB | 19.48 MiB/s, done.
Resolving deltas: 100% (3922/3922), done.


In [5]:
!curl -fsSL https://astral.sh/uv/install.sh | sh

downloading uv 0.8.14 x86_64-unknown-linux-gnu
no checksums to verify
installing to /usr/local/bin
  uv
  uvx
everything's installed!


In [6]:
import max.support.notebook

In [7]:
def save_code_to_file(text: str, filename: str):
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(text)

In [15]:
mojo_code = """
from gpu import thread_idx, block_idx, block_dim, grid_dim, barrier
from os.atomic import Atomic
from gpu.warp import WARP_SIZE
from gpu import block
from gpu.host import DeviceContext
from layout import Layout, LayoutTensor
from layout.tensor_builder import LayoutTensorBuild as tb
from sys import argv
from testing import assert_equal
from math import floor


# ANCHOR: block_sum_dot_product
alias SIZE = 128
alias TPB = 128
alias NUM_BINS = 8
alias in_layout = Layout.row_major(SIZE)
alias out_layout = Layout.row_major(1)
alias dtype = DType.float32

fn traditional_dot_product[
    in_layout: Layout, out_layout: Layout, tpb: Int
](
    output: LayoutTensor[mut=True, dtype, out_layout],
    a: LayoutTensor[mut=False, dtype, in_layout],
    b: LayoutTensor[mut=False, dtype, in_layout],
    size: Int,
):

    shared = tb[dtype]().row_major[tpb]().shared().alloc()
    global_i = block_dim.x * block_idx.x + thread_idx.x
    local_i = thread_idx.x

    # Each thread computes partial product
    if global_i < size:
        a_val = rebind[Scalar[dtype]](a[global_i])
        b_val = rebind[Scalar[dtype]](b[global_i])
        shared[local_i] = a_val * b_val

    barrier()

    # Tree reduction in shared memory - complex but educational
    var stride = tpb // 2
    while stride > 0:
        if local_i < stride:
            shared[local_i] += shared[local_i + stride]
        barrier()
        stride //= 2

    # Only thread 0 writes final result
    if local_i == 0:
        output[0] = shared[0]

fn block_sum_dot_product[
    in_layout: Layout, out_layout: Layout, tpb: Int
](
    output: LayoutTensor[mut=True, dtype, out_layout],
    a: LayoutTensor[mut=False, dtype, in_layout],
    b: LayoutTensor[mut=False, dtype, in_layout],
    size: Int,
):

    global_i = block_dim.x * block_idx.x + thread_idx.x
    local_i = thread_idx.x

    # FILL IN (roughly 6 lines)
    var partial_product: Scalar[dtype] = 0.0
    if global_i < size:
        partial_product = a[global_i][0] * b[global_i][0]
    total = block.sum[block_size=tpb, broadcast=False](val=SIMD[DType.float32, 1](partial_product))
    if local_i == 0:
        output[0] = total[0]

# ANCHOR_END: block_sum_dot_product

# ANCHOR: block_histogram
alias bin_layout = Layout.row_major(SIZE)  # Max SIZE elements per bin


fn block_histogram_bin_extract[
    in_layout: Layout, bin_layout: Layout, out_layout: Layout, tpb: Int
](
    input_data: LayoutTensor[mut=False, dtype, in_layout],
    bin_output: LayoutTensor[mut=True, dtype, bin_layout],
    count_output: LayoutTensor[mut=True, DType.int32, out_layout],
    size: Int,
    target_bin: Int,
    num_bins: Int,
):

    global_i = block_dim.x * block_idx.x + thread_idx.x
    local_i = thread_idx.x

    # Step 1: Each thread determines its bin and element value
    # FILL IN (roughly 9 lines)
    var my_value: Scalar[dtype] = 0.0
    var my_bin: Int = -1

    if global_i < size:
        my_value = input_data[global_i][0]
        my_bin = Int(floor(my_value * num_bins))
        if my_bin >= num_bins:
            my_bin = num_bins - 1
        if my_bin < 0:
            my_bin = 0

    # Step 2: Create predicate for target bin extraction
    # FILL IN (roughly 3 line)
    var belongs_to_target: Int = 0
    if global_i < size and my_bin == target_bin:
        belongs_to_target = 1

    # Step 3: Use block.prefix_sum() for parallel bin extraction!
    # This computes where each thread should write within the target bin
    # FILL IN (1 line)
    write_offset = block.prefix_sum[dtype = DType.int32, block_size=tpb, exclusive=True](val=SIMD[DType.int32, 1](belongs_to_target))

    # Step 4: Extract and pack elements belonging to target_bin
    # FILL IN (roughly 2 line)
    if belongs_to_target == 1:
        bin_output[Int(write_offset[0])] = my_value

    # Step 5: Final thread computes total count for this bin
    # FILL IN (roughly 3 line)
    if local_i == tpb - 1:
        total_count = write_offset[0] + belongs_to_target
        count_output[0] = total_count


# ANCHOR_END: block_histogram

# ANCHOR: block_normalize

alias vector_layout = Layout.row_major(SIZE)


fn block_normalize_vector[
    in_layout: Layout, out_layout: Layout, tpb: Int
](
    input_data: LayoutTensor[mut=False, dtype, in_layout],
    output_data: LayoutTensor[mut=True, dtype, out_layout],
    size: Int,
):

    global_i = block_dim.x * block_idx.x + thread_idx.x
    local_i = thread_idx.x

    # Step 1: Each thread loads its element
    # FILL IN (roughly 3 lines)
    var my_value: Scalar[dtype] = 0.0
    if global_i < size:
        my_value = input_data[global_i][0]

    # Step 2: Use block.sum() to compute total sum (familiar from earlier!)
    # FILL IN (1 line)
    total_sum = block.sum[block_size=tpb, broadcast=False](val=SIMD[DType.float32, 1](my_value))

    # Step 3: Thread 0 computes mean value
    # FILL IN (roughly 4 lines)
    var mean_value: Scalar[dtype] = 1.0
    if local_i == 0:
        if total_sum[0] > 0.0:
            mean_value = total_sum[0] / Float32(size)

    # Step 4: block.broadcast() shares mean to ALL threads!
    # This completes the block operations trilogy demonstration
    # FILL IN (1 line)
    broadcast_mean = block.broadcast[dtype=DType.float32, width=1, block_size=tpb](val=SIMD[DType.float32, 1](mean_value), src_thread=UInt(0))

    # Step 5: Each thread normalizes by the mean
    # FILL IN (roughly 3 lines)
    if global_i < size:
        normalized_value = my_value / broadcast_mean[0]
        output_data[global_i] = normalized_value

# ANCHOR_END: block_normalize


def main():
    if len(argv()) != 2:
        print(
            "Usage: --traditional-dot-product | --block-sum-dot-product |"
            " --histogram | --normalize"
        )
        return

    with DeviceContext() as ctx:
        if argv()[1] == "--traditional-dot-product":
            out = ctx.enqueue_create_buffer[dtype](1).enqueue_fill(0)
            a = ctx.enqueue_create_buffer[dtype](SIZE).enqueue_fill(0)
            b_buf = ctx.enqueue_create_buffer[dtype](SIZE).enqueue_fill(0)

            var expected: Scalar[dtype] = 0.0
            with a.map_to_host() as a_host, b_buf.map_to_host() as b_host:
                for i in range(SIZE):
                    a_host[i] = i
                    b_host[i] = 2 * i
                    expected += a_host[i] * b_host[i]

            print("SIZE:", SIZE)
            print("TPB:", TPB)
            print("Expected result:", expected)

            a_tensor = LayoutTensor[mut=False, dtype, in_layout](a.unsafe_ptr())
            b_tensor = LayoutTensor[mut=False, dtype, in_layout](
                b_buf.unsafe_ptr()
            )
            out_tensor = LayoutTensor[mut=True, dtype, out_layout](
                out.unsafe_ptr()
            )

            # Traditional approach: works perfectly when size == TPB
            ctx.enqueue_function[
                traditional_dot_product[in_layout, out_layout, TPB]
            ](
                out_tensor,
                a_tensor,
                b_tensor,
                SIZE,
                grid_dim=(1, 1),  # ✅ Single block works when size == TPB
                block_dim=(TPB, 1),
            )

            ctx.synchronize()

            with out.map_to_host() as result_host:
                result = result_host[0]
                print("Traditional result:", result)
                assert_equal(result, expected)
                print("Complex: shared memory + barriers + tree reduction")

        elif argv()[1] == "--block-sum-dot-product":
            out = ctx.enqueue_create_buffer[dtype](1).enqueue_fill(0)
            a = ctx.enqueue_create_buffer[dtype](SIZE).enqueue_fill(0)
            b_buf = ctx.enqueue_create_buffer[dtype](SIZE).enqueue_fill(0)

            var expected: Scalar[dtype] = 0.0
            with a.map_to_host() as a_host, b_buf.map_to_host() as b_host:
                for i in range(SIZE):
                    a_host[i] = i
                    b_host[i] = 2 * i
                    expected += a_host[i] * b_host[i]

            print("SIZE:", SIZE)
            print("TPB:", TPB)
            print("Expected result:", expected)

            a_tensor = LayoutTensor[mut=False, dtype, in_layout](a.unsafe_ptr())
            b_tensor = LayoutTensor[mut=False, dtype, in_layout](
                b_buf.unsafe_ptr()
            )
            out_tensor = LayoutTensor[mut=True, dtype, out_layout](
                out.unsafe_ptr()
            )

            # Block.sum(): Same result with dramatically simpler code!
            ctx.enqueue_function[
                block_sum_dot_product[in_layout, out_layout, TPB]
            ](
                out_tensor,
                a_tensor,
                b_tensor,
                SIZE,
                grid_dim=(1, 1),  # Same single block as traditional
                block_dim=(TPB, 1),
            )

            ctx.synchronize()

            with out.map_to_host() as result_host:
                result = result_host[0]
                print("Block.sum result:", result)
                assert_equal(result, expected)
                print("Block.sum() gives identical results!")
                print(
                    "Compare the code: 15+ lines of barriers → 1 line of"
                    " block.sum()!"
                )
                print("Just like warp.sum() but for the entire block")

        elif argv()[1] == "--histogram":
            print("SIZE:", SIZE)
            print("TPB:", TPB)
            print("NUM_BINS:", NUM_BINS)
            print()

            # Create input data with known distribution across bins
            input_buf = ctx.enqueue_create_buffer[dtype](SIZE).enqueue_fill(0)

            # Create test data: values distributed across 8 bins [0.0, 1.0)
            with input_buf.map_to_host() as input_host:
                for i in range(SIZE):
                    # Create values: 0.1, 0.2, 0.3, ..., cycling through bins
                    input_host[i] = (
                        Float32(i % 80) / 100.0
                    )  # Values [0.0, 0.79]

            print("Input sample:", end=" ")
            with input_buf.map_to_host() as input_host:
                for i in range(min(16, SIZE)):
                    print(input_host[i], end=" ")
            print("...")
            print()

            input_tensor = LayoutTensor[mut=False, dtype, in_layout](
                input_buf.unsafe_ptr()
            )

            # Demonstrate histogram for each bin using block.prefix_sum()
            for target_bin in range(NUM_BINS):
                print(
                    "=== Processing Bin",
                    target_bin,
                    "(range [",
                    Float32(target_bin) / NUM_BINS,
                    ",",
                    Float32(target_bin + 1) / NUM_BINS,
                    ")) ===",
                )

                # Create output buffers for this bin
                bin_data = ctx.enqueue_create_buffer[dtype](SIZE).enqueue_fill(
                    0
                )
                bin_count = ctx.enqueue_create_buffer[DType.int32](
                    1
                ).enqueue_fill(0)

                bin_tensor = LayoutTensor[mut=True, dtype, bin_layout](
                    bin_data.unsafe_ptr()
                )
                count_tensor = LayoutTensor[mut=True, DType.int32, out_layout](
                    bin_count.unsafe_ptr()
                )

                # Execute histogram kernel for this specific bin
                ctx.enqueue_function[
                    block_histogram_bin_extract[
                        in_layout, bin_layout, out_layout, TPB
                    ]
                ](
                    input_tensor,
                    bin_tensor,
                    count_tensor,
                    SIZE,
                    target_bin,
                    NUM_BINS,
                    grid_dim=(
                        1,
                        1,
                    ),  # Single block demonstrates block.prefix_sum()
                    block_dim=(TPB, 1),
                )

                ctx.synchronize()

                # Display results for this bin
                with bin_count.map_to_host() as count_host:
                    count = count_host[0]
                    print("Bin", target_bin, "count:", count)

                with bin_data.map_to_host() as bin_host:
                    print("Bin", target_bin, "extracted elements:", end=" ")
                    for i in range(min(8, Int(count))):
                        print(bin_host[i], end=" ")
                    if count > 8:
                        print("...")
                    else:
                        print()
                print()

        elif argv()[1] == "--normalize":
            print("SIZE:", SIZE)
            print("TPB:", TPB)
            print()

            # Create input data with known values for easy verification
            input_buf = ctx.enqueue_create_buffer[dtype](SIZE).enqueue_fill(0)
            output_buf = ctx.enqueue_create_buffer[dtype](SIZE).enqueue_fill(0)

            # Create test data: values like [1, 2, 3, 4, 5, ..., 8, 1, 2, 3, ...]
            # Mean value will be 4.5, so normalized values will be input[i] / 4.5
            var sum_value: Scalar[dtype] = 0.0
            with input_buf.map_to_host() as input_host:
                for i in range(SIZE):
                    # Create values cycling 1-8, mean will be 4.5
                    value = Float32(
                        (i % 8) + 1
                    )  # Values 1, 2, 3, 4, 5, 6, 7, 8, 1, 2, ...
                    input_host[i] = value
                    sum_value += value

            var mean_value = sum_value / Float32(SIZE)

            print("Input sample:", end=" ")
            with input_buf.map_to_host() as input_host:
                for i in range(min(16, SIZE)):
                    print(input_host[i], end=" ")
            print("...")
            print("Sum value:", sum_value)
            print("Mean value:", mean_value)
            print()

            input_tensor = LayoutTensor[mut=False, dtype, in_layout](
                input_buf.unsafe_ptr()
            )
            output_tensor = LayoutTensor[mut=True, dtype, vector_layout](
                output_buf.unsafe_ptr()
            )

            # Execute vector normalization kernel
            ctx.enqueue_function[
                block_normalize_vector[in_layout, vector_layout, TPB]
            ](
                input_tensor,
                output_tensor,
                SIZE,
                grid_dim=(1, 1),  # Single block demonstrates block.broadcast()
                block_dim=(TPB, 1),
            )

            ctx.synchronize()

            # Verify results
            print("Mean Normalization Results:")
            with output_buf.map_to_host() as output_host:
                print("Normalized sample:", end=" ")
                for i in range(min(16, SIZE)):
                    print(output_host[i], end=" ")
                print("...")

                # Verify that the mean normalization worked (mean of output should be ~1.0)
                var output_sum: Scalar[dtype] = 0.0
                for i in range(SIZE):
                    output_sum += output_host[i]

                var output_mean = output_sum / Float32(SIZE)
                print("Output sum:", output_sum)
                print("Output mean:", output_mean)
                print(
                    "✅ Success: Output mean is",
                    output_mean,
                    "(should be close to 1.0)",
                )
        else:
            print(
                "Available options: [--traditional-dot-product |"
                " --block-sum-dot-product | --histogram | --normalize]"
            )

"""

In [16]:
save_code_to_file(mojo_code, "/content/mojo-gpu-puzzles/problems/p27/p27.mojo")

In [17]:
!cd /content/mojo-gpu-puzzles && uv run poe p27 --normalize

Poe => mojo problems/p27/p27.mojo --normalize
SIZE: 128
TPB: 128

Input sample: 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 ...
Sum value: 576.0
Mean value: 4.5

Mean Normalization Results:
Normalized sample: 0.22222222 0.44444445 0.6666667 0.8888889 1.1111112 1.3333334 1.5555556 1.7777778 0.22222222 0.44444445 0.6666667 0.8888889 1.1111112 1.3333334 1.5555556 1.7777778 ...
Output sum: 128.0
Output mean: 1.0
✅ Success: Output mean is 1.0 (should be close to 1.0)


In [11]:
!uv run mojo --version

Mojo 25.4.0 (fbeca2fa)
